In [1]:
import os
import json
import random
import numpy as np
# from PIL import Image # ★ PIL 제거
import cv2 # ★ OpenCV 임포트 추가

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from tqdm import tqdm
import copy
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 # ToTensorV2 임포트 추가 (관례상)

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ================================
# 3. Transform / Dataset / DataLoader (★ Albumentations로 통일 ★)
# ================================
# ResNet 기본 입력 크기 224x224 기준

# ★★★ 수정 1: transform -> train_transform 이름 변경 및 A.Compose 유지 ★★★
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

# ★★★ 수정 2: val_transform도 A.Compose로 통일 (증강 기법 제거) ★★★
val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])


# ★ 경로 확인! ImageFolder의 루트는 'cropped' 폴더가 되어야 합니다. ★
data_root = "/workspace/user4/segmented_output_bisenet"

# ImageFolder 사용: 폴더 이름(anger, happy, panic, sadness)이 클래스 라벨이 됩니다.
train_full_dataset = datasets.ImageFolder(os.path.join(data_root, "train"), transform=None) # transform은 CustomBinaryDataset에서 직접 처리
val_full_dataset = datasets.ImageFolder(os.path.join(data_root, "val"), transform=None) # transform은 CustomBinaryDataset에서 직접 처리

# ImageFolder는 모든 폴더를 클래스로 인식합니다.
# "invalid" 폴더는 제외하고 "happy" vs "others" 2진 분류를 위한 데이터셋 필터링 필요
class_to_idx = train_full_dataset.class_to_idx
target_classes = ["anger", "happy", "panic", "sadness"]

# ⭐️ 2진 분류를 위한 데이터 필터링 ⭐️
def filter_and_remap_dataset(full_dataset, transform, target_classes): # transform 인자 그대로 사용
    
    # 0: OTHERS (anger, panic, sadness), 1: HAPPY
    new_samples = []
    # ImageFolder의 samples는 (파일 경로, 인덱스) 튜플입니다.
    for path, class_idx in full_dataset.samples:
        class_name = full_dataset.classes[class_idx]
        
        # 'invalid' 폴더에 속하는 파일은 건너뜁니다.
        if class_name not in target_classes:
            continue
            
        # 2진 분류 라벨 할당
        if class_name == "happy":
            binary_label = 1 # HAPPY
        else:
            binary_label = 0 # OTHERS (anger, panic, sadness)
            
        new_samples.append((path, binary_label))
        
    # PyTorch 기본 Dataset으로 대체
    class CustomBinaryDataset(Dataset):
        def __init__(self, samples, transform):
            self.samples = samples
            self.transform = transform
            
        def __len__(self):
            return len(self.samples)
            
        def __getitem__(self, idx):
            path, label = self.samples[idx]
            
            # ★ OpenCV로 이미지 로드 ★
            image = cv2.imread(path)
            
            # 파일이 없거나 유효하지 않은 경우 대비
            if image is None:
                 raise FileNotFoundError(f"Image not found or invalid: {path}")

            # OpenCV는 기본적으로 BGR 순서로 로드하므로, RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR -> RGB 변환

            if self.transform is not None:
                # Albumentations 적용 방식 사용 (A.Compose 전달 시)
                augmented = self.transform(image=image)
                image = augmented['image'] # ToTensorV2로 인해 이미 텐서로 변환됨
            
            # 라벨은 torch.long 타입으로 반환
            return image, torch.tensor(label, dtype=torch.long)

    print(f"Original {len(full_dataset)} samples reduced to {len(new_samples)} (excluding invalid).")
    # full_dataset.transform 대신 새로 정의된 transform 사용
    return CustomBinaryDataset(new_samples, transform) 


# ★★★ 수정 3: filter_and_remap_dataset 호출 시 train_transform/val_transform 전달 ★★★
train_dataset = filter_and_remap_dataset(train_full_dataset, train_transform, target_classes) 
val_dataset = filter_and_remap_dataset(val_full_dataset, val_transform, target_classes) 


batch_size = 32
# num_workers=4는 환경에 따라 에러가 발생할 수 있어 0 또는 2로 조정하는 것이 안전할 수 있습니다.
# num_workers = 4 
num_workers = 4 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers)

print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")
print("\n✅ 전처리 및 데이터 로드 완료 (OpenCV + Albumentations). 다음 블록으로 진행하세요.")
# ================================
# 4. 모델 정의 (ResNet18, 2-class) - 변경 없음
# ...
# ================================
# 사전학습된 resnet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 마지막 fc를 2클래스로 교체 (HAPPY vs OTHERS)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

model = model.to(device)
# 가중치 없는 기본 CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


# ================================
# 5. 학습 함수 - 변경 없음
# ...
# ================================
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        # ⭐️ torch.inference_mode() 적용 ⭐️
        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_corrects.double() / val_total
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_resnet_happy_vs_others.pth")
            print(">> best model updated")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


# ================================
# 6. 학습 실행
# ================================
num_epochs = 10
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)


# ================================
# 7. 성능 지표 (val셋을 평가용으로 사용)
# ================================
model.eval()
all_labels = []
all_preds = []

# # ⭐️ torch.inference_mode() 적용 ⭐️
with torch.inference_mode():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, pos_label=1)
rec  = recall_score(all_labels, all_preds, pos_label=1)
f1   = f1_score(all_labels, all_preds, pos_label=1)
cm   = confusion_matrix(all_labels, all_preds)

print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
print(classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"]))

device: cuda
Original 9535 samples reduced to 9535 (excluding invalid).
Original 1149 samples reduced to 1149 (excluding invalid).
Train samples: 9535
Val samples: 1149

✅ 전처리 및 데이터 로드 완료 (OpenCV + Albumentations). 다음 블록으로 진행하세요.

Epoch 1/10
------------------------------


100%|██████████| 298/298 [01:20<00:00,  3.72it/s]

Train Loss: 0.1442 Acc: 0.9448


Val   Loss: 0.2168 Acc: 0.9260
>> best model updated

Epoch 2/10
------------------------------


100%|██████████| 298/298 [01:16<00:00,  3.88it/s]

...

Train Loss: 0.0155 Acc: 0.9945


Val   Loss: 0.0971 Acc: 0.9695

Epoch 10/10
------------------------------


100%|██████████| 298/298 [01:49<00:00,  2.72it/s]

Train Loss: 0.0111 Acc: 0.9959


Val   Loss: 0.2493 Acc: 0.9469

Best val Acc: 0.9748

=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Accuracy : 0.9748
Precision: 0.9655
Recall   : 0.9365
F1-score : 0.9508

Confusion Matrix (rows: true, cols: pred)
[[840  10]
 [ 19 280]]

Classification report:
              precision    recall  f1-score   support

      OTHERS       0.98      0.99      0.98       850
       HAPPY       0.97      0.94      0.95       299

    accuracy                           0.97      1149
   macro avg       0.97      0.96      0.97      1149
weighted avg       0.97      0.97      0.97      1149

